In [0]:
%tensorflow_version 2.x


In [2]:
!pip install --upgrade git+https://github.com/EmGarr/kerod.git


  Cloning https://github.com/EmGarr/kerod.git to /tmp/pip-req-build-tuzptmtd
  Running command git clone -q https://github.com/EmGarr/kerod.git /tmp/pip-req-build-tuzptmtd
  Created wheel for kerod: filename=kerod-0.1-cp36-none-any.whl size=57448 sha256=9430a8cf0a017b9fb8563497655610a7a54e7fba843ac6b8245a4084fb8631b7
  Stored in directory: /tmp/pip-ephem-wheel-cache-uim9h6qb/wheels/4e/e2/2e/00c5521ab8effb4afc054c840e63d6eb0d0004327b5cc83f81
Successfully built kerod


# Download Pascal VOC

Download and preprocess Pascal VOC to the following format (required by od networks):

```python
dataset = {
        'images' : A tensor of float32 and shape [1, height, widht, 3],
        'images_info': A tensor of float32 and shape [1, 2] ,
        'bbox': A tensor of float32 and shape [1, num_boxes, 4],
        'labels': A tensor of int32 and shape [1, num_boxes],
        'num_boxes': A tensor of int32 and shape [1, 1],
        'weights': A tensor of float32 and shape [1, num_boxes]
    }

```

In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds
from kerod.core.standard_fields import BoxField, DatasetField
from kerod.dataset.preprocessing import preprocess

def get_dataset():
  padded_shape = ({
      DatasetField.IMAGES: [None, None, 3],
      DatasetField.IMAGES_INFO: [2]
  },
  {
      BoxField.BOXES: [None, 4],
      BoxField.LABELS: [None],
      BoxField.NUM_BOXES: [1],
      BoxField.WEIGHTS: [None]
  })
  batch_size = 8 # Number of TPU cores
  ds_train, ds_info = tfds.load(name="voc", split="train", shuffle_files=True, with_info=True)
  ds_train = ds_train.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  ds_train =  ds_train.padded_batch(batch_size, padded_shape)
  ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

  ds_test = tfds.load(name="voc", split="test", shuffle_files=False)
  ds_test = ds_test.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  ds_test =  ds_test.padded_batch(batch_size, padded_shape)
  ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)
  return ds_train, ds_test, ds_info

# Load and train the network


In [5]:
import os
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.71.105.34:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.71.105.34:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [6]:
from kerod.core.standard_fields import BoxField
from kerod.core.learning_rate_schedule import WarmupLearningRateScheduler
from kerod.model import factory

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

ds_train, ds_test, ds_info = get_dataset()
# Number of classes of Pascal Voc
classes = ds_info.features['labels'].names
num_classes = len(classes)

with strategy.scope(): 
  model_faster_rcnn = factory.build_model(num_classes)
  base_lr = 0.02
  optimizer = tf.keras.optimizers.SGD(learning_rate=base_lr)
  model_faster_rcnn.compile(optimizer=optimizer, loss=None)
  
callbacks = [WarmupLearningRateScheduler(base_lr, 1, epochs=[8, 10], init_lr=0.0001), TensorBoard(), ModelCheckpoint('.checkpoints/')]

model_faster_rcnn.fit(ds_train, validation_data=ds_test, epochs=11, callbacks=callbacks)

94494720/94487776 [==============================] - 11s 0us/step
Epoch 1/11


ValueError: ignored

# Tensorboard

In [0]:
# Load TENSORBOARD
%load_ext tensorboard
# Start TENSORBOARD
%tensorboard --logdir logs